In [1]:
from random import *
import os
from tqdm import tqdm
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import PIL
import tensorflow as tf
import numpy as np
from keras import layers
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.optimizers import SGD
from keras import Sequential
import warnings
import math
import sys
import gc

In [2]:
class CustomCallback(tf.keras.callbacks.Callback):
    #Base on EalryStoppingAtMinLoss
    def __init__(self,patience=0):
        super(CustomCallback, self).__init__()
        self.patience = patience
        self.bset_weight = None
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait +=1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch")
                self.model.set_weights(self.best_weights)
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" %(self.stopped_epoch +1))

In [16]:
model = VGG16(weights='imagenet')
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])


In [4]:
path = "/media/2/Network/Imagenet_dup/train/"
file_loc = "n03443371/n03443371_10050.JPEG"

In [5]:
!ls /media/2/Network/Imagenet_dup/train/n03443371/n03443371_10050.JPEG

/media/2/Network/Imagenet_dup/train/n03443371/n03443371_10050.JPEG


In [6]:
from tensorflow.keras.applications.vgg19 import preprocess_input
img = image.load_img(path+file_loc,target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

In [7]:
path = "/media/2/Network/extracted_feature/whole_shuffle_to_19/with_error/"
list_train_data = [path+"train_feature_0.npy",path+"train_feature_400000.npy",path+"train_feature_800000.npy"]
list_train_label = [path+"train_label_0.npy",path+"train_label_400000.npy",path+"train_label_800000.npy"]
train_data = []
train_label = []
for train,label in zip(list_train_data, list_train_label):
    train_data.append(np.load(train,mmap_mode="r"))
    train_label.append(np.load(label,mmap_mode="r"))
test_data = np.load(path+"testing_features.npy",mmap_mode="r")
test_label = np.load(path+"testing_label.npy",mmap_mode="r")

In [18]:
index =0
for data,label in zip(train_data,train_label):
    index = index +1
    print("===============",index,"===============")
    model.fit(data,label, epochs=1, verbose=1,
                             use_multiprocessing=True, validation_split=0.2,
                             batch_size=64,callbacks=CustomCallback())

=============== 1 ===============


ValueError: in user code:

    /home/hwbae0326/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/hwbae0326/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/hwbae0326/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/hwbae0326/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/hwbae0326/.local/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/hwbae0326/.local/lib/python3.6/site-packages/tensorflow/py